# Descriptors

Короткое описание в [доке](https://docs.python.org/3/reference/datamodel.html#invoking-descriptors)

Должны быть методы `__get__`, `__set__`, `__del__`

Обычно когда в Python мы хотим получить доступ к атрибуту класса, то сначала мы ищем его в `__dict__` инстанса класса, потом в классе, и далее выше по цепочке наследования.

In [ ]:
class BestConstant:
    def __get__(self, obj, objtype=None):
        print('Using descriptor!')
        return 73

class Number:
    x = 10
    y = BestConstant()  # our descriptor

In [ ]:
num = Number()

In [ ]:
num.x, num.y

Using descriptor!


(10, 73)

In [ ]:
num.__dict__

{}

In [ ]:
Number.__dict__

mappingproxy({'__module__': '__main__',
              'x': 10,
              'y': <__main__.BestConstant at 0x7fe58bc7dfa0>,
              '__dict__': <attribute '__dict__' of 'Number' objects>,
              '__weakref__': <attribute '__weakref__' of 'Number' objects>,
              '__doc__': None})

https://docs.python.org/3/library/types.html#types.MappingProxyType


In [ ]:
num.__dict__['y'] = 42

In [ ]:
num.y

42

In [ ]:
num.y = 32

In [ ]:
num.y

32

Видим, что мы переписали значение. Дескриптор, у которого определен только get, называется non-data дескриптором. https://docs.python.org/3/howto/descriptor.html - больше теории про дескрипторы.

Пример с weakref - слабая ссылка

In [59]:
import weakref

class MyClass:
    def __init__(self, name):
        self.name = name
    def __repr__(self):
        return f"MyClass({self.name})"
obj = MyClass("example")
weak_obj = weakref.ref(obj)
print("Before deletion:", weak_obj())
del obj

# Как и ожидалось слабая ссылка не держит обьект и позволяет его очистить сборщику мусора
# удобно использовать для кеша, что бы избежать переполнения памяти
print("After deletion:", weak_obj())


Before deletion: MyClass(example)
After deletion: None


---

In [133]:
class BestConstantComplete:
    def __get__(self, obj, owner):
        print('Using descriptor!')
        print(f'{owner}')
        return obj._y

    def __set__(self, obj, value):
        print('Setting descriptor!')
        obj._y = value

    def __del__(self, obj):
        del obj._y


class Number:
    y = BestConstantComplete()  # our new descriptor

    def __init__(self, x: int = 10, y: int = 42):
        self.x = x
        self.y = y

In [136]:
num = Number()

Setting descriptor!


In [135]:
num.__dict__

{'x': 10, '_y': 42}

In [17]:
num.y

Using descriptor!
<class '__main__.Number'>


42

In [18]:
num.y = 73

In [19]:
num.y

Using descriptor!
<class '__main__.Number'>


73

In [20]:
num.__dict__

{'x': 10, '_y': 73}

In [21]:
num.__dict__['y'] = 100

In [22]:
num.__dict__

{'x': 10, '_y': 73, 'y': 100}

In [23]:
num.y

Using descriptor!
<class '__main__.Number'>


73

Если у дескриптора определен `__set__`, то Python при попытке достать атрибут по названию, будет доставать сначала дескриптор, даже если в `__dict__` объекта лежит что-то одноименное

---

In [74]:
import os

In [76]:
class DirectorySize:

    def __get__(self, obj, objtype=None):
        return len(os.listdir(obj.dirname))

class Directory:
    size = DirectorySize()

    def __init__(self, dirname):
        self.dirname = dirname
        if not os.path.exists(dirname):
            os.makedirs(dirname)

In [77]:
! ls

sample_data


In [78]:
a = Directory('data')

b = Directory('img')
c = Directory('img/1')

In [79]:
a.size

0

In [80]:
b.size

1

In [81]:
! touch data/tempfile

In [82]:
a.size

1

Дескриптор вызывается при обращении к атрибуту *size*. При этом, код дескриптора выполняется каждый раз

---

**Класс Nuts**:

можно создавать из чего угодно

содержит любой аттрибут

можно удалять и присваивать любые поля

можно доставать по индексу

присваивать значения по индексу

по нeму можно итерироваться

имеет "красивое"  строковое представление

имеет формальное строковое представление

In [219]:
class Nuts:
  def __init__(self, **kwargs):
    self.args = kwargs

  def __repr__(self):
    return f"Nuts({self.args})"

  def __str__(self):
    return f"str({self.args})"

  def __getitem__(self, item):
    return item

  def __getattr__(self, item):
    return item

  def __iter__(self):
    return iter(self.args)






In [221]:
nuts = Nuts(a='34', b='34')
nuts

Nuts({'a': '34', 'b': '34'})

In [222]:
for v in nuts:
  print(v)

a
b


In [215]:
nuts.hhh

'hhh'

---

В чем могут быть проблемы при наследовании от стандартных типов

In [178]:
class DoubleDict(dict):
  def __setitem__(self, key, value):
    super().__setitem__(key, value * 2)

  def update(self, d: dict):
    for key, val in d.items():
      self.__setitem__(key, val)

In [179]:
double_d = DoubleDict()

In [180]:
double_d['sdf'] = 2
double_d

{'sdf': 4}

In [181]:
double_d.update({'afaf': 24, '41': 12})
double_d

{'sdf': 4, 'afaf': 48, '41': 24}

Проигнорили setitem при выполнении update

In [ ]:
class AnswerDict(dict):
    def __getitem__(self, key):
        return 42

In [ ]:
ad = AnswerDict(a='answer')

In [ ]:
ad['a']

42

In [ ]:
simple_dict = {}
simple_dict.update(ad)

In [ ]:
simple_dict

{'a': 'answer'}

Проигнорили getitem, переорпеделение не меняет данные, просто выводит константу

Так произошло потому, что реализация метода update не использует измененные методы. Пофиксить можно наследованием от "пользовательского словаря" из модуля collections.

In [ ]:
from collections import UserDict

In [ ]:
class DoubleDict(UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, value * 2)

In [ ]:
double_d = DoubleDict()

In [ ]:
double_d['sdf'] = 2
double_d

{'sdf': 4}

In [ ]:
double_d.update({'afaf': 24, '41': 12})
double_d

{'sdf': 4, 'afaf': 48, '41': 24}

In [ ]:
class AnswerDict(UserDict):
    def __getitem__(self, key):
        return 42

In [ ]:
ad = AnswerDict(a='answer')

In [ ]:
ad['a']

42

In [ ]:
ad.__dict__

{'data': {'a': 'answer'}}

In [ ]:
simple_dict = {}
simple_dict.update(ad)

In [ ]:
simple_dict

{'a': 42}

---

Выше мы видели, что можем объявить логику итерирования задав только один метод

In [185]:
class Digits:
    digits = '0123456789'

    def __getitem__(self, i):
        return self.digits[i]

#     def __setitem__(self, i, value):  # не будет работать, потому что строки все еще неизменяемые
#         self.digits[i] = value

In [186]:
digits = Digits()

In [187]:
for d in digits: print(d)

0
1
2
3
4
5
6
7
8
9


In [188]:
digits[1] = '0'

TypeError: 'Digits' object does not support item assignment

In [189]:
len(digits)

TypeError: object of type 'Digits' has no len()

Получается, что итерироваться можем, а длину не знаем.. Если вдруг передадим такой класс в код, который ожидает последовательность, можно получить ошибку в неожиданный момент

Можно задать явно недостающий метод

In [ ]:
class Digits:

    def __init__(self, items:str):
        self.digits = items

    def __getitem__(self, i):
        return self.digits[i]

    def __len__(self):
        return len(self.digits)

#     def __setitem__(self, i, value):
#         self.digits[i] = value

In [ ]:
fixed_digits = Digits('4817491724971')

In [ ]:
for i in fixed_digits:
    print(i)

4
8
1
7
4
9
1
7
2
4
9
7
1


In [ ]:
len(fixed_digits)

13

In [ ]:
from random import choice

In [ ]:
choice(fixed_digits)

'4'

---

Также можно добавить в код некоторую проверку на соответсвию интерфейсу. Например, так мы проверяем наш класс на следованию интерфейсу абстрактной последовательности

In [197]:
from collections import abc

class Digits(abc.Sequence):

    digits = '0123456789'

    def __getitem__(self, i):
        return self.digits[i]



In [ ]:
digits = Digits()

TypeError: Can't instantiate abstract class Digits with abstract method __len__

И получаем ошибку при создании объекта, потому что без операции длины это не последовательность в полном смысле

In [199]:
class Digits(abc.Sequence):

    digits = '0123456789'

    def __getitem__(self, i):
        return self.digits[i]

    def __len__(self):
        return len(self.digits)

In [200]:
digits = Digits()

In [201]:
len(digits)

10

https://docs.python.org/3/library/abc.html
@abstractmethod

In [204]:
from abc import ABC, abstractmethod

class Interface(ABC):

  @abstractmethod
  def method(self):
    pass

class Instance(Interface):
  def method(self):
    pass

inst = Instance()